In [1]:
from skimage.transform import resize
from glob import glob
from IPython.display import Audio
import matplotlib.pyplot as plt
import numpy as np
#import librosa
import warnings
import os

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# For use in Google Colab

def use_google_colab(switch):
    if switch:
        from google.colab import files
        uploaded = files.upload()

use_google_colab(False)

In [ ]:
tracks = glob('*.mp3')
tracks

In [ ]:
def chop_bars(track, beats, bar_length=4, beat_step=True):
  """
  Chops up a track into bars of a given length, and will return
  a list of bars. If beat_step=True it will append bars starting
  at every beat.
  """
  bars = []
  beat_idx = librosa.frames_to_samples(beats)

  if beat_step:
    for i in range(len(beat_idx) - bar_length):
      beat_start = beat_idx[i]
      beat_end = beat_idx[i+bar_length]
      sample = track[beat_start: beat_end]
      bars.append(sample)

  else:
    for i in range(0, len(beat_idx) - bar_length, bar_length):
      beat_start = beat_idx[i]
      beat_end = beat_idx[i+bar_length]
      sample = track[beat_start: beat_end]
      bars.append(sample)

  return bars

def loop_sample(sample, loop_count=4):
  tmp = sample
  for i in range(loop_count - 1):
    sample = np.concatenate((tmp, sample))

  return sample

def resize_bars(bars):
  """
  Resize each bar to the average length of all bars.
  Return a numpy array of all bars.
  """
  avg_length = int(np.mean([bar.shape[0] for bar in bars]))
  return np.array([ resize(bar, (avg_length,), anti_aliasing=True) for bar in bars ])

In [ ]:
bars_chopped_4beats = []
bars_chopped_8beats = []
bars_chopped_16beats = []

bars_stepped_4beats = []
bars_stepped_8beats = []
bars_stepped_16beats = []

for track in tracks:
  print(track)
  audio_array, sr = librosa.load(track)
  tempo, beats = librosa.beat.beat_track(y=audio_array, sr=sr)

  #bars_chopped_4beats += chop_bars(audio_array, beats, bar_length=4, beat_step=False)
  #bars_chopped_8beats += chop_bars(audio_array, beats, bar_length=8, beat_step=False)
  #bars_chopped_16beats += chop_bars(audio_array, beats, bar_length=16, beat_step=False)
  #bars_stepped_4beats += chop_bars(audio_array, beats, bar_length=4, beat_step=True)
  #bars_stepped_8beats += chop_bars(audio_array, beats, bar_length=8, beat_step=True)
  bars_stepped_16beats += chop_bars(audio_array, beats, bar_length=16, beat_step=True)


In [ ]:
print(f"4 beat chopped bars: {len(bars_chopped_4beats)}")
print(f"8 beat chopped bars: {len(bars_chopped_8beats)}")
print(f"16 beat chopped bars: {len(bars_chopped_16beats)}")
print(f"4 beat stepped bars: {len(bars_stepped_4beats)}")
print(f"8 beat stepped bars: {len(bars_stepped_8beats)}")
print(f"16 beat stepped bars: {len(bars_stepped_16beats)}")

In [ ]:
#bars_chopped_4beats_resized = resize_bars(bars_chopped_4beats)
#bars_chopped_8beats_resized = resize_bars(bars_chopped_8beats)
#bars_chopped_16beats_resized = resize_bars(bars_chopped_16beats)
#bars_stepped_4beats_resized = resize_bars(bars_stepped_4beats)
#bars_stepped_8beats_resized = resize_bars(bars_stepped_8beats)
bars_stepped_16beats = resize_bars(bars_stepped_16beats)

In [ ]:
#print(f"4 beat chopped bars shape: {bars_chopped_4beats_resized.shape}")
#print(f"8 beat chopped bars shape: {bars_chopped_8beats_resized.shape}")
#print(f"16 beat chopped bars shape: {bars_chopped_16beats_resized.shape}")
#print(f"4 beat stepped bars shape: {bars_stepped_4beats_resized.shape}")
#print(f"8 beat stepped bars shape: {bars_stepped_8beats_resized.shape}")
print(f"16 beat stepped bars shape: {bars_stepped_16beats_resized.shape}")

In [ ]:
clip = bars_stepped_16beats_resized[-150]
print(clip.shape)

loop = loop_sample(clip, loop_count=2)

Audio(loop, rate=22500)

In [ ]:
np.savez_compressed("daft_punk_clips_chopped_4beats.npz",X=bars_chopped_4beats_resized)

In [ ]:
np.savez_compressed("daft_punk_clips_chopped_8beats.npz",X=bars_chopped_8beats_resized)

In [ ]:
np.savez_compressed("daft_punk_clips_chopped_16beats.npz",X=bars_chopped_16beats_resized)

In [ ]:
np.savez_compressed("daft_punk_clips_stepped_4beats.npz",X=bars_stepped_4beats_resized)

In [ ]:
np.savez_compressed("daft_punk_clips_stepped_8beats.npz",X=bars_stepped_8beats_resized)

In [ ]:
np.savez_compressed("daft_punk_clips_stepped_16beats.npz",X=bars_stepped_16beats_resized)